In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

In [2]:
data  = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
data.head()
Col = list(data.columns)
Col.remove('Personal Loan')
Col.append('Personal Loan')
data = data[Col].drop(columns='ZIP Code')

In [3]:
from sklearn.model_selection import train_test_split
X = data.values[:,:-1]
X = X.reshape(X.shape[0],1,X.shape[1])
Y = data.values[:,-1].reshape(X.shape[0],1)
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25)

In [4]:
w_inp = np.random.rand(1,12)
w_hid = np.random.rand(1,4)

In [5]:
def Sigmoid(x):
    s = 1 / (1+np.exp(-1*x))
    return s

In [6]:
def dri_Sigmoid(x):
    s = Sigmoid(x) * (1-Sigmoid(x))
    return s

In [7]:
def Predict(S,x_test,y_test):
    from sklearn.metrics import accuracy_score
    wI = S['w_inp']
    wH = S['w_hid']
    inp_out = (wI*x_test)
    inp_out = inp_out.reshape(inp_out.shape[0],4,3).sum(axis=2)
    inp_out = Sigmoid(inp_out)
    hid_out = wH*inp_out
    hid_out = hid_out.reshape(hid_out.shape[0],1,4).sum(axis=2)
    hid_out = np.floor(Sigmoid(hid_out)) 
    
    return accuracy_score(hid_out,y_test)*100

In [8]:
def D1toMat(sol):
    Z = {
        "w_inp" : (sol.reshape(4,4)[:3]).reshape(1,12),
        "w_hid" : sol.reshape(4,4)[-1]
    }
    return Z

In [9]:
def Fitness(sol,X,Y):
    fitness =[]
    for i in sol :
        Z = D1toMat(i)
        fitness.append(Predict(Z,X,Y))

    return fitness

In [10]:
#Parameters For PSO
Gbest = 0 
w_aso = np.random.rand()
r1 =  np.random.rand()
r2 = np.random.rand()
c1 = np.random.rand()
c2 = np.random.rand()

In [11]:
def Updater(DF_PSO,x_test,y_test):
    return Fitness(list(DF_PSO['Weights'].values),x_test,y_test) 

In [12]:
def Global(prev,data):
    data = np.array(data)
    mtemp = data.max()
    if(prev>mtemp):
        return prev
    else:
        return mtemp

In [13]:
# generating Population
DF_PSO = pd.DataFrame()
Pop = []
pop_size = 100
for i in range(pop_size):
    w_inp = np.random.rand(1,12)
    w_hid = np.random.rand(1,4)
    temp = np.append(w_inp,w_hid,axis=1)
    Pop.append(temp)
DF_PSO['Weights']  = Pop
# Generating Velocity for each sol
V = []
for i in range(pop_size):
    temp = np.random.rand(1,16)
    V.append(temp)
DF_PSO['Velocity'] = V
DF_PSO['Pbest'] = np.array(Updater(DF_PSO,x_test,y_test))*0.01

In [14]:
def PSO(DF_PSO,pop_size,x_test,y_test,Gbest,w_aso,c1,c2,r1,r2,epoch=50):
    for i in tqdm(range(epoch)):
        Velocity = DF_PSO['Velocity'].values
        Weights = DF_PSO['Weights'].values
        Pbest = DF_PSO['Pbest'].values.reshape(pop_size,1)
        Velocity = (w_aso * Velocity) + c1*r1*(Pbest -Weights ) + c2*r2*(Gbest-Weights)
        Weights = Weights + Velocity
        DF_PSO['Velocity'] = Velocity
        DF_PSO['Weights'] = Weights
        DF_PSO['Pbest'] = np.array(Updater(DF_PSO,x_test,y_test))*0.01
    DF_PSO.sort_values(by='Pbest',ascending=False)
    Best = D1toMat(DF_PSO['Weights'][0])
    Fit = DF_PSO['Pbest'][0]
    return Best,Fit*100
    

In [15]:
Gbest = Global(Gbest,DF_PSO['Pbest'])
PSO(DF_PSO,pop_size,x_test,y_test,Gbest,w_aso,c1,c2,r1,r2,100)

100%|██████████| 100/100 [00:10<00:00,  9.90it/s]


({'w_inp': array([[0.8968, 0.8968, 0.8968, 0.8968, 0.8968, 0.8968, 0.8968, 0.8968,
          0.8968, 0.8968, 0.8968, 0.8968]]),
  'w_hid': array([0.8968, 0.8968, 0.8968, 0.8968])},
 89.68)

# END